## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-14-58-52 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/21/us/sup...
1,2026-01-21-14-57-42 +0000,nyt,"After Maduro’s Last Dance, a New Rhythm Goes V...",https://www.nytimes.com/2026/01/21/world/ameri...
2,2026-01-21-14-57-17 +0000,nyt,How a Pro-Greenland Song Became a Protest Anthem,https://www.nytimes.com/2026/01/21/arts/music/...
3,2026-01-21-14-56-44 +0000,nyt,"For Trump, the Supreme Court Case Reflects Goa...",https://www.nytimes.com/2026/01/21/us/politics...
4,2026-01-21-14-56-02 +0000,nyt,Filoplumes Are Nature’s Super Feather,https://www.nytimes.com/2026/01/20/science/bir...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2369/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,94
10,greenland,40
70,davos,17
4,new,17
253,world,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
40,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,198
12,2026-01-21-14-45-01 +0000,startribune,Trump in Davos says NATO should allow the US t...,https://www.startribune.com/trump-is-on-his-wa...,191
177,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,177
205,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,172
155,2026-01-21-06-42-52 +0000,bbc,Trump doubles down on Greenland ahead of Davos...,https://www.bbc.com/news/articles/cvgr19m642zo...,168


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
40,198,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...
229,66,2026-01-20-23-45-18 +0000,latimes,State GOP seeks Supreme Court injunction to bl...,https://www.latimes.com/california/story/2026-...
202,63,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
99,50,2026-01-21-10-51-01 +0000,bbc,UK inflation rises for first time in five mont...,https://www.bbc.com/news/articles/cnvg251g25yo...
380,48,2026-01-20-16-06-20 +0000,nypost,Treasury Secretary Scott Bessent lashes out at...,https://nypost.com/2026/01/20/business/trumps-...
70,46,2026-01-21-11-51-26 +0000,nyt,Man Gets Life in Prison for Killing Shinzo Abe...,https://www.nytimes.com/2026/01/20/world/asia/...
356,44,2026-01-20-17-53-33 +0000,nypost,Iran warns that attack on Ayatollah ​Ali Khame...,https://nypost.com/2026/01/20/world-news/iran-...
270,40,2026-01-20-22-12-28 +0000,nyt,Minnesota Police Leaders Criticize Federal Tac...,https://www.nytimes.com/2026/01/20/us/ice-minn...
105,40,2026-01-21-10-30-00 +0000,wsj,Google searches for “No Buy January” have hit ...,https://www.wsj.com/economy/consumers/no-buy-j...
122,39,2026-01-21-09-55-53 +0000,bbc,Train driver killed in second deadly Spain rai...,https://www.bbc.com/news/articles/c1m78xl0gmpo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
